# Tutorial

http://zzz.bwh.harvard.edu/plink/tutorial.shtml

Once the .zip file is downloaded, type `./plink` from the command line. 

One can see [how to use the command line](http://zzz.bwh.harvard.edu/plink/download.shtml#input) to run plink.

To add the executable to the PATH, copy it to the `/usr/local/bin` or `~/bin`, assuming those directories are in the path. Use `$PATH` command to see which dirs are into it.

In [2]:
!$PATH

/bin/sh: 1: /home/matthias/anaconda3/bin:/home/matthias/anaconda3/bin:/home/matthias/anaconda3/condabin:/home/matthias/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin: not found


# Running PLINK from command line

Use commands to produce summary statistics on missing data, exclude some SNPs, run an association analysis. Any command runs a new instanciation of plink.

For example, if you want to run two association tests with different PED files, but include SNPs that are above a certain MAF in both runs: 
[...]

# Tutorial example

Randomly selected genotypes (about 80'000 autosomal SNPs), from the 89 Asian HapMap individuals. A phenotype has been simulated based on the genotype at one SNP. 

The example data has been put into the `data/` directory. It contins the genotypes, map files and two extra phenotype files. Two phenotypes were generated :

* Quantitative trait
* Disease trait (1=unaffected, 2=affected)

The quantitative trait was generated as a function of three components:
* Random comp
* Chinese vs Japanese identity
* Variant on chromosome 2, rs2222162

One can see te contingency table representing the joint distribution of the disease and subpopulation. It shows a strong relationship between these two variables. 

Another contingency table shows association between the variant and the disease which also shows a strong association. 

**In summary**, there is a single causal variant that is associated with the disease. But there are complicating factors, this is one among many SNPs, and there might be some confounding of the SNP-disease associations due to the subpopulation-disease association - i.e. that *population stratification* effect exist. Even though we consider the two populations are similar, but we still need to take this into account. 

We'll use the affection status variable as the default variable for analysis (sixth column in the PED file). The quantitative trait is in a separate alternate phenotype file, `qt.phe`. The `pop.phe` contains a dummy phenotype (1=Chinese, 2=Japanese). We'll use this to investigate between-population differences. Here we focus on autosomal SNPs for simplicity

## Analyse the data

### Getting started

Type plink and specifying a file without options -> check if the file is intact. To get basic summary statistics: 
    
    plink --file hapmap1

In [9]:
# The analysis was runned in a terminal
# Here we display the content of the .log file
!cat /media/matthias/Data/Programmation/HBV/tutorial/hapmap1/plink.log


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ plink.log ]
Analysis started: Mon Aug  5 14:12:54 2019

Options in effect:
	--file hapmap1
	--noweb

83534 (of 83534) markers to be included from [ hapmap1.map ]
89 individuals read from [ hapmap1.ped ] 
89 individuals with nonmissing phenotypes
Assuming a disease phenotype (1=unaff, 2=aff, 0=miss)
Missing phenotype value is also -9
44 cases, 45 controls and 0 missing
89 males, 0 females, and 0 of unspecified sex
Before frequency and genot

The `--file` option takes as a single parameter the root of the input file names. It looks for the *PED* and *MAP* files. That means `--file hapmap1` implies that `hapmap1.ped` and `hapmap1.map` exist in the current directory. 

* PED files contain genotype information
* MAP files contain information on the name and position of the markers in the PED file

Consult [this page (data)](http://zzz.bwh.harvard.edu/plink/data.shtml) to get familiar with those file formats.  

### Content of the log file

1. Copyright and version number, web-checking
1. Message indicating that everything is in the .log file
1. Commands used
1. Information on the number of markers and individuals from the MAP and PED files. 
1. Information on the phenotype and affection status variable (as opposed to a quantitative trait), shows the missing values (here -9).
1. Filtering stage: individuals and/or SNPs are removed on the basis of thresholds. In this case, no individuals were removed, but almost 20'000 SNPs were removed based on missingness (859) and frequency (16994). 
1. Finally, a line indicates that the analysis is finished. 

### Making a binary PED file

This is a more compact representation of the data, it saves space and speed for subsequent analysis. Use the command

    plink --file hapmap1 --make-bed --out hapmap1
    
* `--make-bed` option automatically exclude nothing from data 
* Three files are created: binary file that contains the raw genotype data `hapmap1.bed`, but also a revised map file `hapmap1.bim` which contains two extra columns that give the allele names for each SNP, and `hapmap1.fam` which is just the first six columns of `hapmap1.ped`.
* You can view the `.bim` and `.fam` files but not the `.bed` files. 

Example: create binary files that only include individuals with high genotyping (95% complete at least) :

In [15]:
!plink --noweb --file tutorial/hapmap1/hapmap1 --make-bed --mind 0.05 --out tutorial/hapmap1/highgeno


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ tutorial/hapmap1/highgeno.log ]
Analysis started: Mon Aug  5 14:34:45 2019

Options in effect:
	--noweb
	--file tutorial/hapmap1/hapmap1
	--make-bed
	--mind 0.05
	--out tutorial/hapmap1/highgeno

83534 (of 83534) markers to be included from [ tutorial/hapmap1/hapmap1.map ]
89 individuals read from [ tutorial/hapmap1/hapmap1.ped ] 
89 individuals with nonmissing phenotypes
Assuming a disease phenotype (1=unaff, 2=aff, 0=miss)
Missing phenot

### Working with binary PED file

Simply use `--bfile` instead of `--file`. From this point you can actually delete the `hapmap1.ped` and `hapmap1.map` files, since the binary option will make plink load the files `hapmap1.bim`, `hapmap1.fam`, `hapmap1.bed`. 

#### Summary statistics: missing rates

Using `--missing` option. 

     plink --bfile hapmap1 --missing --out miss_stat 

It shows the **removed individuals based on missingness**. For example, MIND > 0.1 implies tht we accept people with less than 10% missingness.

The per individual and per SNP (after exclusion) rates are output to the files `miss_stat.imiss` and `miss_stat.lmiss` respectively. 

In [21]:
!head -10 tutorial/hapmap1/miss_stat.lmiss

 CHR         SNP   N_MISS   N_GENO   F_MISS
   1   rs6681049        0       89        0
   1   rs4074137        0       89        0
   1   rs7540009        0       89        0
   1   rs1891905        0       89        0
   1   rs9729550        0       89        0
   1   rs3813196        0       89        0
   1   rs6704013        2       89  0.02247
   1    rs307347       12       89   0.1348
   1   rs9439440        2       89  0.02247


For each SNP *(because .lmiss)*, we see the number of individuals `N_MISS` and the proportion of missing individuals `F_MISS`. 

In [22]:
!head -10 tutorial/hapmap1/miss_stat.imiss

     FID  IID MISS_PHENO   N_MISS   N_GENO   F_MISS
  HCB181    1          N      671    83534 0.008033
  HCB182    1          N     1156    83534  0.01384
  HCB183    1          N      498    83534 0.005962
  HCB184    1          N      412    83534 0.004932
  HCB185    1          N      329    83534 0.003939
  HCB186    1          N     1233    83534  0.01476
  HCB187    1          N      258    83534 0.003089
  HCB188    1          N      864    83534  0.01034
  HCB189    1          N      517    83534 0.006189


The final column is the actual genotyping rate for that individual (we see that it's very high here).

### Summary statistics: allele frequencies

Similar analysis, except requesting allele frequencies instead of genotyping rates. We'll generate a `freq_stat.frq` file which contains the minor allele frequency and allele codes for each SNP. 

    plink --bfile hapmap1 --freq --out freq_stat

In [23]:
!head -10 tutorial/hapmap1/freq_stat.frq

 CHR         SNP   A1   A2          MAF  NCHROBS
   1   rs6681049    1    2       0.2135      178
   1   rs4074137    1    2      0.07865      178
   1   rs7540009    0    2            0      178
   1   rs1891905    1    2       0.4045      178
   1   rs9729550    1    2       0.1292      178
   1   rs3813196    1    2      0.02809      178
   1   rs6704013    0    2            0      174
   1    rs307347    0    2            0      154
   1   rs9439440    0    2            0      174


We can also perform this frequency analysis (and missingness analysis) stratified by categorical, cluster variable. In this case, indicate whether the individual is Chinese or Japanese. The cluster files will contain three columns, each row is an individual. See the [documentation](http://zzz.bwh.harvard.edu/plink/data.shtml#cluster) for more information about the format.

    # Stratified analysis of allele frequency
    plink --bfile hapmap1 --freq --within pop.phe --out freq_stat
    
This generates a file `freq_stat.frq.strat` instead of `freq_stat.frq` :

In [25]:
!head -10 tutorial/hapmap1/freq_stat.frq.strat

 CHR         SNP     CLST   A1   A2      MAF    MAC  NCHROBS
   1   rs6681049        1    1    2   0.2333     21       90 
   1   rs6681049        2    1    2   0.1932     17       88 
   1   rs4074137        1    1    2      0.1      9       90 
   1   rs4074137        2    1    2  0.05682      5       88 
   1   rs7540009        1    0    2        0      0       90 
   1   rs7540009        2    0    2        0      0       88 
   1   rs1891905        1    1    2   0.4111     37       90 
   1   rs1891905        2    1    2   0.3977     35       88 
   1   rs9729550        1    1    2   0.1444     13       90 


We see each row is now the allele frequency for each SNP stratifed by subpopulation. Each SNP is represented twice: the CLST column indicates whether the frequency is from the Chinese or Japanese populations, coded as in the `pop.phe` file. 

If you're interested in a specific SNP, use te `--snp` option. 

In [29]:
!plink --noweb --bfile tutorial/hapmap1/hapmap1 --snp rs1891905 --freq --within tutorial/hapmap1/pop.phe --out tutorial/hapmap1/snp1_frq_stat


@----------------------------------------------------------@
|        PLINK!       |     v1.07      |   10/Aug/2009     |
|----------------------------------------------------------|
|  (C) 2009 Shaun Purcell, GNU General Public License, v2  |
|----------------------------------------------------------|
|  For documentation, citation & bug-report instructions:  |
|        http://pngu.mgh.harvard.edu/purcell/plink/        |
@----------------------------------------------------------@

Skipping web check... [ --noweb ] 
Writing this text to log file [ tutorial/hapmap1/snp1_frq_stat.log ]
Analysis started: Mon Aug  5 15:57:38 2019

Options in effect:
	--noweb
	--bfile tutorial/hapmap1/hapmap1
	--snp rs1891905
	--freq
	--within tutorial/hapmap1/pop.phe
	--out tutorial/hapmap1/snp1_frq_stat

Reading map (extended format) from [ tutorial/hapmap1/hapmap1.bim ] 
83534 markers to be included from [ tutorial/hapmap1/hapmap1.bim ]
Scan region on chromosome 1 from [ rs1891905 ] to [ rs1891905 ]
R

In [32]:
!cat tutorial/hapmap1/snp1_frq_stat.frq.strat

 CHR         SNP     CLST   A1   A2      MAF    MAC  NCHROBS
   1   rs1891905        1    1    2   0.4111     37       90 
   1   rs1891905        2    1    2   0.3977     35       88 


This shows the population-specific frequencies for this single SNP. See [this link](http://zzz.bwh.harvard.edu/plink/dataman.shtml#extract) to see specific options to select a range of SNPs (`--window kb`, `--from`, `--to` options ...).

### Basic association analysis

Perform a basic association analysis on the disease trait for all single SNPs.

    plink --bfile hapmap1 --assoc --out as1

In [33]:
!head tutorial/hapmap1/as1.assoc

 CHR         SNP         BP   A1      F_A      F_U   A2        CHISQ            P           OR 
   1   rs6681049          1    1   0.1591   0.2667    2        3.067      0.07991       0.5203 
   1   rs4074137          2    1  0.07955  0.07778    2     0.001919       0.9651        1.025 
   1   rs7540009          3    0        0        0    2           NA           NA           NA 
   1   rs1891905          4    1   0.4091      0.4    2      0.01527       0.9017        1.038 
   1   rs9729550          5    1   0.1705  0.08889    2        2.631       0.1048        2.106 
   1   rs3813196          6    1  0.03409  0.02222    2       0.2296       0.6318        1.553 
   1   rs6704013          7    0        0        0    2           NA           NA           NA 
   1    rs307347          8    0        0        0    2           NA           NA           NA 
   1   rs9439440          9    0        0        0    2           NA           NA           NA 


Each row is a single SNP association result. The columns are : 

* Chromosome
* SNP identifier
* Code for allele 1 (the minor, rare allele based on the entire sample frequencies)
* Frequency of this variant in cases
* Frequency of this variant in controls
* Code or the other allele
* Chi-squared statistic for this test (1 df)
* Asymptotic significance value for this test
* OR (odds ratio) for this test

If a test isn't defined (for example the variant is monomorphic but not excluded), the value `NA` is given. For Linux environment, simply sort them and print out the top 10:

***
**WARNING : the output is not exactly as in the tutorial.. Should investigate where does it come from.** Thus prefer the plink commands to sort by significance. 

In [41]:
!sort --key=6 -nr tutorial/hapmap1/as1.assoc > tutorial/hapmap1/as1_sorted.assoc
!head tutorial/hapmap1/as1_sorted.assoc

  13   rs9314912      61931    2   0.3523   0.6333    1        14.06     0.000177       0.3149 
   2   rs2222162      10602    1   0.2841   0.6222    2        20.51    5.918e-06       0.2409 
   1    rs513287       4350    2   0.3523   0.6222    1        12.98    0.0003155       0.3302 
  14   rs3825726      66182    2   0.3523   0.6222    1        12.98    0.0003155       0.3302 
   7  rs12705332      37957    1    0.375   0.6111    2        9.923     0.001632       0.3818 
   3   rs7621054      15360    2   0.3636   0.6111    1        10.91    0.0009588       0.3636 
   1  rs12028453       3183    2   0.3523   0.6111    1        11.94    0.0005501       0.3461 
  14   rs4899962      66836    2   0.3023   0.6111    1        16.88    3.983e-05       0.2758 
   9   rs2118021      47713    1    0.375   0.6023    2        9.096     0.002562       0.3962 
  15   rs1912010      69505    1   0.3864   0.6023    2        8.206     0.004176       0.4158 


***
Here you see that the simulated disease variant rs2222162 is the secont most significant. It has a large difference in allele frequencies of 0.28 in cases versus 0.62 in controls. But there is another SNP on chr13 that has a slightly higher  test result. We'll investigate whether this is due to chance alone or because of some confounding factors due to population stratification. 

The important thing to remember is that when so many tests are performed, we often expect the distribution of true positive results to be virtually indistinguishable from the  best false positive results. 

To get a sorted list of association results, including a range of significance adjusted for multiple comparisons, use the `--adjust` flag:

    plink --bfile hapmap1 --assoc --adjust --out as2
    
This generates the file `as2.assoc.adjust` in addition to the basic `as2.assoc` file.

In [44]:
!head tutorial/hapmap1/as2.assoc.adjusted

 CHR         SNP      UNADJ         GC       BONF       HOLM   SIDAK_SS   SIDAK_SD     FDR_BH     FDR_BY
  13   rs9585021  5.586e-06  4.994e-05     0.3839     0.3839     0.3188     0.3188    0.09719          1 
   2   rs2222162  5.918e-06  5.232e-05     0.4068     0.4067     0.3342     0.3342    0.09719          1 
   9  rs10810856  7.723e-06  6.483e-05     0.5308     0.5308     0.4118     0.4118    0.09719          1 
   2   rs4675607   8.05e-06  6.703e-05     0.5533     0.5533     0.4249     0.4249    0.09719          1 
   2   rs4673349  8.485e-06  6.994e-05     0.5832     0.5831     0.4419     0.4419    0.09719          1 
   2   rs1375352  8.485e-06  6.994e-05     0.5832     0.5831     0.4419     0.4419    0.09719          1 
  21    rs219746  1.228e-05  9.422e-05     0.8442     0.8441     0.5701     0.5701     0.1206          1 
   1   rs4078404  2.667e-05   0.000176          1          1     0.8401       0.84     0.2291          1 
  14   rs1152431  3.862e-05  0.0002374         

This is a pre-sorted list of association results which the following fields:

* Chromosome
* SNP id
* Unadjusted, asymptotic significance value
* Genomic control adjusted significance value. The estimation is based on the inflation factor based on the median chi2 statistic. These values do not control for multiple testing.
* Bonferroni adjusted significance value
* Holm step-down adjusted significance value
* Sidak single-step adjusted signficiance value
* Sidak step-down adjusted sig value
* ... step-up FDR control
* ... step-up FDR control

**In this case, we see that no single variant is significant at the 5% level after genome-wide correction.** Note that different correction measures have different properties (not discussed here). 

# Python and PLINK
https://pypi.org/project/pandas-plink/

Must install a package for reading PLINK binary file format. 

## Install

    pip install pandas-plink
    # aternatively
    conda install -c conda-forge pandas-plink

## Usage

    from pandas_plink import read_plink

In [19]:
from pandas_plink import read_plink1_bin
G = read_plink1_bin("tutorial/hapmap1/hapmap1.bed", 
                    "tutorial/hapmap1/hapmap1.bim",
                    "tutorial/hapmap1/hapmap1.fam")

Mapping files: 100%|██████████| 3/3 [00:00<00:00, 10.20it/s]


In [20]:
print(G)

<xarray.DataArray 'genotype' (sample: 89, variant: 83534)>
dask.array<shape=(89, 83534), dtype=float64, chunksize=(89, 1024)>
Coordinates:
  * sample   (sample) object '1' '1' '1' '1' '1' '1' ... '1' '1' '1' '1' '1' '1'
  * variant  (variant) object '1_rs6681049' '1_rs4074137' ... '22_rs756638'
    father   (sample) <U1 '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    fid      (sample) <U6 'HCB181' 'HCB182' 'HCB183' ... 'JPT268' 'JPT269'
    gender   (sample) <U1 '1' '1' '1' '1' '1' '1' ... '1' '1' '1' '1' '1' '1'
    iid      (sample) <U1 '1' '1' '1' '1' '1' '1' ... '1' '1' '1' '1' '1' '1'
    mother   (sample) <U1 '0' '0' '0' '0' '0' '0' ... '0' '0' '0' '0' '0' '0'
    trait    (sample) float64 1.0 1.0 2.0 1.0 1.0 1.0 ... 1.0 2.0 2.0 2.0 2.0
    a0       (variant) <U1 '1' '1' '0' '1' '1' '1' ... '1' '1' '0' '0' '1' '1'
    a1       (variant) <U1 '2' '2' '2' '2' '2' '2' ... '2' '2' '2' '2' '2' '2'
    chrom    (variant) <U2 '1' '1' '1' '1' '1' '1' ... '22' '22' '22' '22' '22'
 